In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt


In [6]:
# Reload the dataset
spotify_df = pd.read_csv("high_popularity_spotify_data.csv")

# Select numerical features for similarity analysis
features = [
    "energy", "danceability", "valence", "acousticness", 
    "speechiness", "instrumentalness", "tempo", "loudness"
]

# Drop rows with missing values in selected features
spotify_df_clean = spotify_df.dropna(subset=features)


NameError: name 'pd' is not defined

In [ ]:
# Normalize features using Min-Max scaling
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(spotify_df_clean[features])

# Add scaled features back into DataFrame for reference
scaled_df = pd.DataFrame(scaled_features, columns=[f"{col}_scaled" for col in features])
spotify_df_scaled = pd.concat([spotify_df_clean.reset_index(drop=True), scaled_df], axis=1)

# Set up query indices (choosing 3 distinct songs)
query_indices = [1, 3, 5]
queries = spotify_df_scaled.iloc[query_indices]

In [ ]:
# Compute pairwise distances
dist_matrix = euclidean_distances(scaled_features)

# Find top 10 most similar tracks for each query (excluding itself)
similar_tracks = {}
for idx in query_indices:
    distances = dist_matrix[idx]
    similar_indices = np.argsort(distances)[1:11]  # skip index 0 (itself)
    similar_tracks[idx] = spotify_df_scaled.iloc[similar_indices][["track_artist", "energy", "danceability", "valence", "loudness"]]

# Package the results for display
query_results = {
    spotify_df_scaled.iloc[idx]["track_artist"]: similar_tracks[idx].reset_index(drop=True)
    for idx in query_indices
}